# Les projets Open Source de Machine Learning sont-ils fait par des chercheurs ?

## Préambule

Nous observons l'apparition fréquente de nouveaux algorithmes de Machine Learning et leur intégration à des bibliothèques Open Source.

Nous souhaitons évaluer les hypothèses suivantes :
